# 02 - Embeddings con Transformers

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2-2.5 horas

---

## Índice

1. [De Word2Vec a Transformers](#intro)
2. [BERT: El primer Transformer contextual](#bert)
3. [Sentence Transformers](#sentence)
   - 3.1 Modelos Open Source
   - 3.2 Búsqueda semántica
4. [Visualización con UMAP](#umap)
5. [Embeddings de proveedores comerciales](#comercial)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Comprender la diferencia entre embeddings estáticos y contextuales
- Utilizar BERT y Sentence Transformers para generar embeddings
- Implementar búsqueda semántica con similitud coseno
- Visualizar embeddings en 3D con UMAP
- Conocer las opciones comerciales disponibles

<a name="intro"></a>
## 1. De Word2Vec a Transformers

### El problema de los embeddings estáticos

En el notebook anterior vimos Word2Vec y GloVe. Estos modelos tienen una limitación fundamental: **generan un único vector por palabra**, sin importar el contexto.

**Ejemplo del problema:**
- "Voy al **banco** a sacar dinero" (institución financiera)
- "Me senté en el **banco** del parque" (asiento)
- "Vimos un **banco** de peces" (grupo de animales)

En Word2Vec, las tres oraciones usarían el **mismo vector** para "banco", perdiendo el significado contextual.

### Evolución histórica

| Modelo | Año | Tipo | Características |
|--------|-----|------|----------------|
| Word2Vec | 2013 | Estático | Un vector por palabra |
| ELMo | 2018 | Contextual (BiLSTM) | Primer modelo contextual |
| BERT | 2018 | Contextual (Transformer) | Bidireccional, atención |
| Sentence-BERT | 2019 | Sentence-level | Embeddings de oraciones |

## Instalación de dependencias

Todas las librerías usadas son **gratuitas y open source**.

In [1]:
# Install required libraries (all free and open source)
!pip install -q transformers sentence-transformers torch
!pip install -q umap-learn plotly scikit-learn
!pip install -q tf-keras

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\frane\\anaconda3\\Lib\\site-packages\\~l_dtypes\\_ml_dtypes_ext.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
# Standard imports
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo: {device}")

Dispositivo: cpu


<a name="bert"></a>
## 2. BERT: El primer Transformer contextual

### ¿Qué es BERT?

**BERT** (Bidirectional Encoder Representations from Transformers) fue desarrollado por Google en 2018. Sus características principales son:

- **Bidireccional**: Lee toda la oración simultáneamente (no izquierda-a-derecha)
- **Contextual**: Genera diferentes embeddings según el contexto
- **Pre-entrenado**: Entrenado en grandes corpus, ajustable para tareas específicas

### Arquitectura de BERT

| Modelo | Capas | Heads | Parámetros | Dimensión |
|--------|-------|-------|------------|----------|
| BERT-Base | 12 | 12 | 110M | 768 |
| BERT-Large | 24 | 16 | 340M | 1024 |

### Tareas de pre-entrenamiento

1. **Masked Language Modeling (MLM)**: Predecir palabras enmascaradas
   - Entrada: "El [MASK] ladra fuerte"
   - Salida: "perro"

2. **Next Sentence Prediction (NSP)**: ¿Es B la siguiente oración de A?
   - A: "El cliente hizo el pedido."
   - B: "El sistema registró el pago." → [IsNext]

In [3]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model (English)
print("Cargando BERT base (inglés)...")
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
model_bert = BertModel.from_pretrained("bert-base-uncased")
model_bert.eval()  # Set to evaluation mode

print(f"Modelo cargado: bert-base-uncased")

ImportError: numpy>=1.17,<2.0 is required for a normal functioning of this module, but found numpy==2.2.6.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [ ]:
# Example: Get BERT embeddings
text = "The bank approved my loan application"

# Tokenize and get embeddings
tokens = tokenizer_bert(text, return_tensors="pt", padding=True)

with torch.no_grad():
    outputs = model_bert(**tokens)

# Get embeddings from last hidden state
embeddings = outputs.last_hidden_state

print(f"Texto: '{text}'")
print(f"Tokens: {tokenizer_bert.convert_ids_to_tokens(tokens['input_ids'][0])}")
print(f"Shape de embeddings: {embeddings.shape}")
print(f"  - Batch size: {embeddings.shape[0]}")
print(f"  - Número de tokens: {embeddings.shape[1]}")
print(f"  - Dimensión del embedding: {embeddings.shape[2]}")

### BERT en español

Existen modelos BERT entrenados específicamente en español. Uno de los más populares es **BETO**.

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load Spanish BERT model (BETO)
print("Cargando BETO (BERT español)...")
modelo_id = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer_es = AutoTokenizer.from_pretrained(modelo_id)
model_es = AutoModel.from_pretrained(modelo_id)
model_es.eval()

print(f"Modelo cargado: {modelo_id}")

In [ ]:
# Test with Spanish text
texto_es = "El banco central ajustó las tasas de interés"

tokens_es = tokenizer_es(texto_es, return_tensors="pt")
with torch.no_grad():
    outputs_es = model_es(**tokens_es)

embeddings_es = outputs_es.last_hidden_state

print(f"Texto: '{texto_es}'")
print(f"Tokens: {tokenizer_es.convert_ids_to_tokens(tokens_es['input_ids'][0])}")
print(f"Shape de embeddings: {embeddings_es.shape}")

### Comparación de modelos

| Modelo | Idiomas | Embeddings Contextuales | Bidireccional | Subpalabras |
|--------|---------|------------------------|---------------|-------------|
| Word2Vec | Mono | ❌ | ❌ | ❌ |
| GloVe | Mono | ❌ | ❌ | ❌ |
| ELMo | Mono | ✅ | Parcial | ❌ |
| BERT | Multi | ✅ | ✅ | ✅ |

<a name="sentence"></a>
## 3. Sentence Transformers

### El problema de BERT para oraciones

BERT genera un embedding **por cada token**. Para obtener un embedding de toda la oración, necesitamos agregar estos vectores (promedio, CLS token, etc.), lo cual no es óptimo.

### ¿Qué es Sentence Transformers?

**Sentence Transformers** es una librería construida sobre HuggingFace que:
- Genera un **único vector** por oración/párrafo
- Optimizado para **similitud semántica**
- Perfecto para búsqueda semántica, clustering, clasificación

### Modelos populares

| Modelo | Idiomas | Dimensiones | Uso recomendado |
|--------|---------|-------------|----------------|
| all-MiniLM-L6-v2 | Multi | 384 | Rápido, buena precisión |
| all-mpnet-base-v2 | Multi | 768 | Mejor precisión |
| paraphrase-multilingual-MiniLM-L12-v2 | Multi | 384 | Multilingüe |
| hiiamsid/sentence_similarity_spanish_es | ES | 768 | Español específico |

In [ ]:
from sentence_transformers import SentenceTransformer

# Load Sentence Transformer model (free, open source)
print("Cargando modelo Sentence Transformer...")
model_st = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print(f"Modelo cargado: all-MiniLM-L6-v2")
print(f"Dimensión del embedding: 384")

In [ ]:
# Generate sentence embedding
sentence = "Artificial intelligence is transforming the world."
embedding = model_st.encode(sentence)

print(f"Oración: '{sentence}'")
print(f"Tipo: {type(embedding)}")
print(f"Shape: {embedding.shape}")
print(f"Primeros 10 valores: {embedding[:10]}")

### 3.2 Búsqueda Semántica

Una de las aplicaciones más importantes de Sentence Transformers es la **búsqueda semántica**: encontrar documentos similares a una consulta basándose en el significado, no solo en palabras clave.

In [ ]:
from sentence_transformers import util

# Query and candidate sentences
query = "¿Cómo puedo abrir una cuenta bancaria?"

candidates = [
    "Para abrir una cuenta bancaria, visita una sucursal con tu DNI.",
    "Las cuentas bancarias se abren en línea fácilmente.",
    "El banco central bajó los tipos de interés.",
    "Me gusta el helado de vainilla.",
    "Este es un buen banco para sentarse.",
    "Ayer vi nadando en el mar un banco de peces.",
    "Los requisitos incluyen identificación y comprobante de domicilio.",
    "El clima está muy agradable hoy."
]

print(f"Consulta: '{query}'")
print(f"\nCandidatos: {len(candidates)} oraciones")

In [ ]:
# Encode query and candidates
query_embedding = model_st.encode(query, convert_to_tensor=True)
candidate_embeddings = model_st.encode(candidates, convert_to_tensor=True)

# Calculate cosine similarity
similarities = util.cos_sim(query_embedding, candidate_embeddings)[0]

# Sort by similarity
results = sorted(zip(candidates, similarities.tolist()), 
                key=lambda x: x[1], reverse=True)

print("Resultados ordenados por similitud semántica:")
print("=" * 60)
for sentence, score in results:
    emoji = "✅" if score > 0.4 else "❌"
    print(f"{emoji} {score:.4f} | {sentence}")

### Observaciones de la búsqueda semántica

Nota cómo el modelo:
- **Entiende el contexto**: Relaciona "abrir cuenta" con "requisitos" e "identificación"
- **Distingue homónimos**: Diferencia "banco" (financiero) de "banco" (asiento) y "banco" (peces)
- **Ignora irrelevantes**: Baja puntuación para oraciones sin relación semántica

In [ ]:
# Function for semantic search
def semantic_search(query, documents, model, top_k=5):
    """
    Perform semantic search on documents.
    
    Parameters:
    - query: search query string
    - documents: list of document strings
    - model: SentenceTransformer model
    - top_k: number of results to return
    
    Returns:
    - list of (document, score) tuples
    """
    query_emb = model.encode(query, convert_to_tensor=True)
    doc_embs = model.encode(documents, convert_to_tensor=True)
    
    similarities = util.cos_sim(query_emb, doc_embs)[0]
    
    # Get top-k indices
    top_indices = similarities.argsort(descending=True)[:top_k]
    
    results = [(documents[i], similarities[i].item()) for i in top_indices]
    return results

# Test the function
new_query = "What documents do I need for banking?"
results = semantic_search(new_query, candidates, model_st, top_k=3)

print(f"Query: '{new_query}'\n")
for doc, score in results:
    print(f"  {score:.4f} | {doc}")

<a name="umap"></a>
## 4. Visualización con UMAP

**UMAP** (Uniform Manifold Approximation and Projection) es un algoritmo de reducción de dimensionalidad que preserva mejor la estructura local que PCA, ideal para visualizar embeddings.

In [ ]:
import umap
import plotly.express as px

# Sample sentences for visualization
sentences = [
    # Animals
    "The dog barks in the garden",
    "The cat sleeps on the couch",
    "The bird sings in the morning",
    "The horse runs in the field",
    # Technology
    "Artificial intelligence is advancing rapidly",
    "Machine learning requires lots of data",
    "Neural networks learn patterns automatically",
    "Deep learning powers modern AI applications",
    # Finance
    "The stock market crashed yesterday",
    "Banks offer different interest rates",
    "Investment requires careful planning",
    "The economy is recovering slowly",
    # Food
    "Pizza is my favorite food",
    "I love eating sushi for dinner",
    "The restaurant serves excellent pasta",
    "Cooking at home is healthier"
]

categories = (
    ["Animals"] * 4 + 
    ["Technology"] * 4 + 
    ["Finance"] * 4 + 
    ["Food"] * 4
)

print(f"Total de oraciones: {len(sentences)}")
print(f"Categorías: {set(categories)}")

In [ ]:
# Generate embeddings
embeddings_viz = model_st.encode(sentences)
print(f"Shape de embeddings: {embeddings_viz.shape}")

In [ ]:
# Apply UMAP for 3D visualization
reducer = umap.UMAP(n_components=3, random_state=42, n_neighbors=5)
embeddings_3d = reducer.fit_transform(embeddings_viz)

print(f"Shape después de UMAP: {embeddings_3d.shape}")

In [ ]:
# 3D visualization with Plotly
fig = px.scatter_3d(
    x=embeddings_3d[:, 0],
    y=embeddings_3d[:, 1],
    z=embeddings_3d[:, 2],
    text=sentences,
    color=categories,
    title="Embeddings de oraciones (Sentence-BERT + UMAP 3D)",
    labels={'color': 'Categoría'}
)

fig.update_traces(marker=dict(size=8))
fig.update_layout(
    scene=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3'
    ),
    width=900,
    height=700
)
fig.show()

### Observaciones de la visualización

En el gráfico 3D podemos observar:
- Las oraciones de la misma categoría tienden a agruparse
- UMAP preserva la estructura semántica mejor que PCA
- Los clusters están bien separados

Esto demuestra que los Sentence Transformers capturan efectivamente el significado semántico.

<a name="comercial"></a>
## 5. Embeddings de proveedores comerciales (Opcional)

Además de los modelos open source, existen embeddings comerciales de alta calidad:

### OpenAI Embeddings
| Modelo | Dimensiones | Precio (aprox) |
|--------|-------------|----------------|
| text-embedding-3-small | 1536 | $0.00002/1K tokens |
| text-embedding-3-large | 3072 | $0.00013/1K tokens |

### Google Gemini Embeddings
| Modelo | Dimensiones | Precio |
|--------|-------------|--------|
| text-embedding-004 | 768 | Gratis (límites) |
| gemini-embedding-exp | 768-3072 | Variable |

**Nota:** Para este curso, usamos modelos open source para evitar costos. Los modelos comerciales pueden ofrecer mejor rendimiento en algunos casos.

In [ ]:
# Example code for OpenAI embeddings (requires API key)
# Uncomment and add your API key to use

# from openai import OpenAI
# 
# OPENAI_API_KEY = "your-api-key-here"
# client = OpenAI(api_key=OPENAI_API_KEY)
# 
# response = client.embeddings.create(
#     input="Sample text for embedding",
#     model="text-embedding-3-small"
# )
# 
# embedding = response.data[0].embedding
# print(f"OpenAI embedding dimension: {len(embedding)}")

print("El código para embeddings comerciales está comentado.")
print("Para usarlo, descomenta y añade tu API key.")

<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio 1: Búsqueda semántica en español

In [ ]:
# Exercise 1: Create a semantic search system for FAQs

faqs = [
    "¿Cuál es el horario de atención al cliente?",
    "¿Cómo puedo cambiar mi contraseña?",
    "¿Cuáles son los métodos de pago aceptados?",
    "¿Cómo realizo una devolución de producto?",
    "¿Tienen envío internacional?",
    "¿Cuánto tiempo tarda el envío?",
    "¿Puedo cancelar mi pedido?",
    "¿Cómo contacto con soporte técnico?"
]

# Test queries
test_queries = [
    "Quiero devolver algo que compré",
    "¿Aceptan tarjeta de crédito?",
    "Olvidé mi clave de acceso"
]

print("Sistema de FAQ con búsqueda semántica")
print("=" * 50)

for query in test_queries:
    results = semantic_search(query, faqs, model_st, top_k=2)
    print(f"\nPregunta: '{query}'")
    print("Respuestas más relevantes:")
    for faq, score in results:
        print(f"  {score:.4f} | {faq}")

### Ejercicio 2: Comparar diferentes modelos

In [ ]:
# Exercise 2: Compare embedding dimensions and speed
import time

models_to_compare = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
]

test_texts = [
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with many layers.",
    "Natural language processing enables computers to understand text."
]

print("Comparación de modelos:")
print("=" * 60)

for model_name in models_to_compare:
    print(f"\nModelo: {model_name}")
    model = SentenceTransformer(model_name)
    
    start = time.time()
    embeddings = model.encode(test_texts)
    elapsed = time.time() - start
    
    print(f"  Dimensión: {embeddings.shape[1]}")
    print(f"  Tiempo: {elapsed:.4f}s para {len(test_texts)} textos")

### Ejercicio 3: Clustering semántico

In [ ]:
# Exercise 3: Semantic clustering
from sklearn.cluster import KMeans

mixed_sentences = [
    # Tech (cluster 0)
    "Python is a popular programming language",
    "JavaScript runs in web browsers",
    "Machine learning requires data",
    # Sports (cluster 1)  
    "Football is played with 11 players",
    "Basketball requires a hoop and ball",
    "Tennis is played on a court",
    # Food (cluster 2)
    "Pizza originated in Italy",
    "Sushi is a Japanese dish",
    "Tacos are popular in Mexico"
]

# Get embeddings
embs = model_st.encode(mixed_sentences)

# Apply K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(embs)

print("Clustering semántico (sin etiquetas previas):")
print("=" * 50)

for cluster_id in range(3):
    print(f"\nCluster {cluster_id}:")
    for sent, c in zip(mixed_sentences, clusters):
        if c == cluster_id:
            print(f"  - {sent}")

## Resumen

En este notebook hemos aprendido:

1. **Embeddings contextuales**: BERT genera diferentes vectores según el contexto
2. **Sentence Transformers**: Embeddings optimizados para oraciones completas
3. **Búsqueda semántica**: Encontrar documentos por significado, no palabras
4. **Visualización UMAP**: Reducción de dimensionalidad preservando estructura

### Cuándo usar cada modelo

| Caso de uso | Modelo recomendado |
|-------------|-------------------|
| Búsqueda semántica rápida | all-MiniLM-L6-v2 |
| Multilingüe | paraphrase-multilingual-MiniLM-L12-v2 |
| Máxima precisión | all-mpnet-base-v2 |
| Español específico | BETO o hiiamsid/sentence_similarity_spanish_es |

En el siguiente notebook veremos **Ingeniería de Prompts**, fundamental para trabajar con LLMs.

---

## Referencias

- [BERT Paper (Devlin et al., 2018)](https://arxiv.org/abs/1810.04805)
- [Sentence-BERT Paper (Reimers & Gurevych, 2019)](https://arxiv.org/abs/1908.10084)
- [Sentence Transformers Documentation](https://www.sbert.net/)
- [HuggingFace Transformers](https://huggingface.co/docs/transformers/)